# Custom Interactivity

In [ ]:
import param
import numpy as np
import holoviews as hv
from holoviews import opts

hv.extension('bokeh', 'matplotlib')

In previous notebooks we discovered how the ``DynamicMap`` class allows us to declare objects in a lazy way to enable exploratory analysis of large parameter spaces. In the [Responding to Events](./12-Responding_to_Events.ipynb) guide we learned how to interactively push updates to existing plots by declaring Streams on a DynamicMap. In this user guide we will extend the idea to so called *linked* Streams, which allows complex interactions to be declared by specifying which events should be exposed when a plot is interacted with. By passing information about live interactions to a simple Python based callback, you will be able to build richer, even more interactive visualizations that enable seamless data exploration.

Some of the possibilities this opens up include:

* Dynamically aggregating datasets of billions of datapoints depending on the plot axis ranges using the [datashader](./15-Large_Data.ipynb) library.
* Responding to ``Tap`` and ``DoubleTap`` events to reveal more information in subplots.
* Computing statistics in response to selections applied with box- and lasso-select tools.

Currently only the bokeh backend for HoloViews supports the linked streams system but the principles used should extend to any backend that can define callbacks that fire when a user zooms or pans or interacts with a plot.

<center><div class="alert alert-info" role="alert">To use and visualize <b>DynamicMap</b> with linked <b>Stream</b> objects you need to be running a live Jupyter server.<br>This user guide assumes that it will be run in a live notebook environment.<br>
When viewed statically, DynamicMaps on this page will only show the first available Element.<br></div></center>

## Available Linked Streams

There are a huge number of ways one might want to interact with a plot. The HoloViews streams module aims to expose many of the most common interactions you might want want to employ, while also supporting extensibility via custom linked Streams. 

Here is the full list of linked Stream that are all descendants of the ``LinkedStream`` baseclass:

In [ ]:
from holoviews import streams
listing = ', '.join(sorted([str(s.name) for s in param.descendents(streams.LinkedStream)]))
print('The linked stream classes supported by HoloViews are:\n\n{listing}'.format(listing=listing))

As you can see, most of these events are about specific interactions with a plot such as the current axis ranges (the ``RangeX``, ``RangeY`` and ``RangeXY`` streams), the mouse pointer position (the ``PointerX``, ``PointerY`` and ``PointerXY`` streams), click or tap positions (``Tap``, ``DoubleTap``). Additionally there a streams to access plotting selections made using box- and lasso-select tools (``Selection1D``), the plot size (``PlotSize``) and the ``Bounds`` of a selection. Finally there are a number of drawing/editing streams such as ``BoxEdit``, ``PointDraw``, ``FreehandDraw``, ``PolyDraw`` and ``PolyEdit``.

Each of these linked Stream types has a corresponding backend specific ``Callback``, which defines which plot attributes or events to link the stream to and triggers events on the ``Stream`` in response to changes on the plot. Defining custom ``Stream`` and ``Callback`` types will be covered in future guides.

## Linking streams to plots

At the end of the [Responding to Events](./12-Responding_to_Events.ipynb) guide we discovered that streams have ``subscribers``, which allow defining user defined callbacks on events, but also allow HoloViews to install subscribers that let plots respond to Stream updates. Linked streams add another concept on top of ``subscribers``, namely the Stream ``source``.

The source of a linked stream defines which plot element to receive events from. Any plot containing the ``source`` object will be attached to the corresponding linked stream and will send event values in response to the appropriate interactions.

Let's start with a simple example. We will declare one of the linked Streams from above, the ``PointerXY`` stream. This stream sends the current mouse position in plot axes coordinates, which may be continuous or categorical. The first thing to note is that we haven't specified a ``source`` which means it uses the default value of ``None``.

In [ ]:
pointer = streams.PointerXY()
print(pointer.source)

Before continuing, we can check the stream parameters that are made available to user callbacks from a given stream instance by looking at its contents:

In [ ]:
print('The %s stream has contents %r' % (pointer, pointer.contents))

#### Automatic linking

A stream instance is automatically linked to the first ``DynamicMap`` we pass it to, which we can confirm by inspecting the stream's ``source`` attribute after supplying it to a ``DynamicMap``:

In [ ]:
pointer_dmap = hv.DynamicMap(lambda x, y: hv.Points([(x, y)]), streams=[pointer])
print(pointer.source is pointer_dmap)

The ``DynamicMap`` we defined above simply defines returns a ``Points`` object composed of a single point that marks the current ``x`` and ``y`` position supplied by our ``PointerXY`` stream. The stream is linked whenever this ``DynamicMap`` object is displayed as it is the stream source:

In [ ]:
pointer_dmap.opts(size=10)

If you hover over the plot canvas above you can see that the point tracks the current mouse position. We can also inspect the last cursor position by examining the stream contents:

In [ ]:
pointer.contents

In the [Responding to Events](12-Responding_to_Events.ipynb) user guide, we introduced an integration example that would work more intuitively with linked streams. Here it is again with the ``limit`` value controlled by the ``PointerX`` linked stream:

In [ ]:
xs = np.linspace(-3, 3, 400)

def function(xs, time):
    "Some time varying function"
    return np.exp(np.sin(xs+np.pi/time))

def integral(limit, time):
    limit = -3 if limit is None else np.clip(limit,-3,3)
    curve = hv.Curve((xs, function(xs, time)))[limit:]
    area  = hv.Area ((xs, function(xs, time)))[:limit]
    summed = area.dimension_values('y').sum() * 0.015  # Numeric approximation
    return (area * curve * hv.VLine(limit) * hv.Text(limit + 0.8, 2.0, '%.2f' % summed))

integral_streams = [
    streams.Stream.define('Time', time=1.0)(),
    streams.PointerX().rename(x='limit')]

integral_dmap = hv.DynamicMap(integral, streams=integral_streams)

integral_dmap.opts(
    opts.Area(color='#fff8dc', line_width=2),
    opts.Curve(color='black'),
    opts.VLine(color='red'))

We only needed to import and use the ``PointerX`` stream and rename the ``x`` parameter that tracks the cursor position to 'limit' so that it maps to the corresponding argument. Otherwise, the example only required bokeh specific style options to match the matplotlib example as closely as possible.

#### Explicit linking

In the example above, we took advantage of the fact that a ``DynamicMap`` automatically becomes the stream source if a source isn't explicitly specified. If we want to link the stream instance to a different object we can specify our source explicitly. Here we will create a 2D ``Image`` of sine gratings, and then declare that this image is the ``source`` of the ``PointerXY`` stream. This pointer stream is then used to generate a single point that tracks the cursor when hovering over the image:

In [ ]:
xvals = np.linspace(0,4,202)
ys,xs = np.meshgrid(xvals, -xvals[::-1])
img = hv.Image(np.sin(((ys)**3)*xs))

pointer = streams.PointerXY(x=0,y=0, source=img)
pointer_dmap = hv.DynamicMap(lambda x, y: hv.Points([(x, y)]), streams=[pointer])

Now if we display a ``Layout`` consisting of the ``Image`` acting as the source together with the ``DynamicMap``, the point shown on the right tracks the cursor position when hovering over the image on the left:

In [ ]:
img + pointer_dmap.opts(size=10, xlim=(-.5, .5), ylim=(-.5, .5))

This will even work across different cells. If we use this particular stream instance in another ``DynamicMap`` and display it, this new visualization will also be supplied with the cursor position when hovering over the image. 

To illustrate this, we will now use the pointer ``x`` and ``y`` position to generate cross-sections of the image at the cursor position on the ``Image``, making use of the ``Image.sample`` method. Note the use of ``np.clip`` to make sure the cross-section is well defined when the cusor goes out of bounds:

In [ ]:
x_sample = hv.DynamicMap(lambda x, y: img.sample(x=np.clip(x,-.49,.49)), streams=[pointer])
y_sample = hv.DynamicMap(lambda x, y: img.sample(y=np.clip(y,-.49,.49)), streams=[pointer])

(x_sample + y_sample).opts(opts.Curve(framewise=True))

Now when you hover over the ``Image`` above, you will see the cross-sections update while the point position to the right of the ``Image`` simultaneously updates.

#### Unlinking objects

Sometimes we just want to display an object designated as a source without linking it to the stream. If the object is not a ``DynamicMap``, like the ``Image`` we designated as a ``source`` above, we can make a copy of the object using the ``clone`` method. We can do the same with ``DynamicMap`` though we just need to supply ``link_inputs=False`` as an extra argument.

Here we will create a ``DynamicMap`` that draws a cross-hair at the cursor position:

In [ ]:
pointer = streams.PointerXY(x=0, y=0)
cross_dmap = hv.DynamicMap(lambda x, y: (hv.VLine(x) * hv.HLine(y)), streams=[pointer])

Now we will add two copies of the ``cross_dmap`` into a Layout but the subplot on the right will not be linking the inputs. Try hovering over the two subplots and observe what happens:

In [ ]:
cross_dmap + cross_dmap.clone(link=False)

Notice how hovering over the left plot updates the crosshair position on both subplots, while hovering over the right subplot has no effect.

## Transient linked streams

In the basic  [Responding to Events](12-Responding_to_Events.ipynb) user guide we saw that stream parameters can be updated and those values are then passed to the callback. This model works well for many different types of streams that have well-defined values at all times.

This approach is not suitable for certain events which only have a well defined value at a particular point in time. For instance, when you hover your mouse over a plot, the hover position always has a well-defined value but the click position is only defined when a click occurs (if it occurs).

This latter case is an example of what are called 'transient' streams. These streams are supplied new values only when they occur and fall back to a default value at all other times. This default value is typically ``None`` to indicate that the event is not occuring and therefore has no data.


Transient streams are particularly useful when you are subscribed to multiple streams, some of which are only occasionally triggered. A good example are the ``Tap`` and ``DoubleTap`` streams; while you sometimes just want to know the last tapped position, we can only tell the two events apart if their values are ``None`` when not active. 

We'll start by declaring a ``SingleTap`` and a ``DoubleTap`` stream as ``transient``. Since both streams supply 'x' and 'y' parameters, we will rename the ``DoubleTap`` parameters to 'x2' and 'y2'.

In [ ]:
tap = streams.SingleTap(transient=True)
double_tap = streams.DoubleTap(rename={'x': 'x2', 'y': 'y2'}, transient=True)

Next we define a list of taps we can append to, and a function that accumulates the tap and double tap coordinates along with the number of taps, returning a ``Points`` Element of the tap positions.

In [ ]:
taps = []

def record_taps(x, y, x2, y2):
    if None not in [x,y]:
        taps.append((x, y, 1))
    elif None not in [x2, y2]:
        taps.append((x2, y2, 2))
    return hv.Points(taps, vdims='Taps')

Finally we can create a ``DynamicMap`` from our callback and attach the streams. We also apply some styling so the points are colored depending on the number of taps.

In [ ]:
taps_dmap = hv.DynamicMap(record_taps, streams=[tap, double_tap])

taps_dmap.opts(color='Taps', cmap={1: 'red', 2: 'gray'}, size=10, tools=['hover'])

Now try single- and double-tapping within the plot area, each time you tap a new point is appended to the list and displayed. Single taps show up in red and double taps show up in grey.  We can also inspect the list of taps directly:

In [ ]:
taps